In [1]:
import finnhub
import pandas as pd
import requests
import math
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


API_key = "08a76bb048683631e32c58ecd7741a1d"

In [2]:
key = "bugd5sn48v6s5hg59ba0"
finnhub_client = finnhub.Client(api_key=key)

ticker = "AAPL"

In [3]:

#getting financial data
data = finnhub_client.financials(ticker, 'bs', 'quarterly')
print(finnhub_client.financials(ticker, 'bs', 'quarterly'))

{'financials': [{'accountsPayable': 42296, 'accountsReceivables': 16120, 'accruedLiability': 1436, 'accumulatedDepreciation': -66760, 'cash': 17773, 'cashEquivalents': 20243, 'cashShortTermInvestments': 90943, 'commonStock': 50779, 'currentAssets': 143713, 'currentLiabilities': 105392, 'currentPortionLongTermDebt': 8797, 'deferredIncomeTax': None, 'goodwill': None, 'intangiblesAssets': None, 'inventory': 4061, 'liabilitiesShareholdersEquity': 323888, 'longTermDebt': 98667, 'longTermInvestments': 100887, 'nonRedeemablePreferredStock': None, 'otherCurrentAssets': 11264, 'otherCurrentliabilities': 47867, 'otherEquity': -2252, 'otherLiabilities': 53853, 'otherLongTermAssets': 33952, 'otherReceivables': 21325, 'period': '2020-09-26', 'preferredSharesOutstanding': None, 'propertyPlantEquipment': 45336, 'retainedEarnings': 14966, 'sharesOutstanding': 16976.763, 'shortTermDebt': 4996, 'shortTermInvestments': 52927, 'tangibleBookValueperShare': 3.84873, 'totalAssets': 323888, 'totalDebt': 11309

In [4]:
#sorting and adding year and quarter columns

pd.set_option('display.max_columns', None)
df = pd.DataFrame(data["financials"])
df["year"] = pd.DatetimeIndex(df['period']).year
df = df.sort_values(by='period')

df["year"] = pd.DatetimeIndex(df['period']).year
df.loc[pd.DatetimeIndex(df['period']).month == 3, 'quarter'] = '1Q'
df.loc[pd.DatetimeIndex(df['period']).month == 6, 'quarter'] = '2Q'
df.loc[pd.DatetimeIndex(df['period']).month == 9, 'quarter'] = '3Q'
df.loc[pd.DatetimeIndex(df['period']).month == 12, 'quarter'] = '4Q'
df["yq"] = df["year"].map(str) + "." + df["quarter"].map(str)


df = df.dropna(axis='columns')
df = df.reset_index()
df

index  accountsPayable  accountsReceivables  accumulatedDepreciation  \
0      124              334                  793                     -309   
1      123              259                  749                     -342   
2      122              236                  659                     -378   
3      121              279                  706                     -415   
4      120              341                  762                     -447   
..     ...              ...                  ...                      ...   
120      4            46236                22926                   -58579   
121      3            45111                20970                   -60959   
122      2            32421                15722                   -62854   
123      1            35325                17882                   -65230   
124      0            42296                16120                   -66760   

     cashEquivalents  cashShortTermInvestments  commonStock  currentAssets  \
0                438                       809          315           2294   
1                411                       908          284           2326   
2                646                       996          220           2270   
3                665                      1044          179           2339   
4                375                       997          137           2403   
..               ...                       ...          ...            ...   
120            36640                    100557        45174         162819   
121            28388                    107162        45972         163231   
122            15769                     94051        48032         143753   
123            13130                     93025        48696         140065   
124            20243                     90943        50779         143713   

     currentLiabilities  inventory  liabilitiesShareholdersEquity  \
0                   895        475                           2744   
1                   893        427                           2813   
2                   774        312                           2776   
3                   887        292                           2890   
4                  1027        355                           2976   
..                  ...        ...                            ...   
120              105718       4106                         338516   
121              102161       4097                         340618   
122               96094       3334                         320400   
123               95318       3978                         317344   
124              105392       4061                         323888   

     otherCurrentAssets  otherLongTermAssets  otherReceivables      period  \
0                   217                  115                 0  1989-09-29   
1                   242                  126                 0  1989-12-29   
2                   303                  138                 0  1990-03-30   
3                   298                  172                 0  1990-06-29   
4                   289                  174                 0  1990-09-28   
..                  ...                  ...               ...         ...   
120               12352                32978             22878  2019-09-28   
121               12026                33195             18976  2019-12-28   
122               15691                33868             14955  2020-03-28   
123               10987                32836             14193  2020-06-27   
124               11264                33952             21325  2020-09-26   

     propertyPlantEquipment  retainedEarnings  sharesOutstanding  \
0                       334              1176           3536.000   
1                       362              1248           3506.000   
2                       368              1341           3452.000   
3                       379              1335           3351.000   
4                       398          

In [5]:
quarter_data = finnhub_client.company_revenue_estimates('AAPL', freq='quarterly')
quarter_df = pd.DataFrame(quarter_data['data'])
quarter_df = quarter_df.sort_values(by="period")
quarter_df = quarter_df.astype({'period': 'datetime64'} )
quarter_df


numberAnalysts     period    revenueAvg   revenueHigh   revenueLow
105               3 1996-06-30    2385250000    2397000000   2279500000
104               5 1996-09-30    2424158329    2499000000   2252340000
103               4 1996-12-31    2311662500    2514300000   2027300000
102               4 1997-03-31    2476092500    2524500000   2331880000
101               4 1997-06-30    2591802499    2639760000   2443430000
..              ...        ...           ...           ...          ...
4                22 2021-09-30   74484755402   81182820000  63195272050
3                12 2021-12-31  107081926524  112489680000  99012187400
2                10 2022-03-31   74645539765   80637120000  68155988820
1                10 2022-06-30   68762190142   72228240000  64088870000
0                10 2022-09-30   78865119080   81984540000  71538470000

[106 rows x 5 columns]

In [6]:
quarter_df["year"] = pd.DatetimeIndex(quarter_df['period']).year
quarter_df.loc[pd.DatetimeIndex(quarter_df['period']).month == 3, 'quarter'] = '1Q'
quarter_df.loc[pd.DatetimeIndex(quarter_df['period']).month == 6, 'quarter'] = '2Q'
quarter_df.loc[pd.DatetimeIndex(quarter_df['period']).month == 9, 'quarter'] = '3Q'
quarter_df.loc[pd.DatetimeIndex(quarter_df['period']).month == 12, 'quarter'] = '4Q'
quarter_df["yq"] = quarter_df["year"].map(str) + "." + quarter_df["quarter"].map(str)
quarter_df = quarter_df.reset_index()

quarter_df = quarter_df[["revenueAvg" , "yq"]]

indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
quarter_df["next2QRevenue"] = quarter_df["revenueAvg"].rolling(window=indexer, min_periods=1).sum()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=4)
quarter_df["next4QRevenue"] = quarter_df["revenueAvg"].rolling(window=indexer, min_periods=1).sum()
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=6)
quarter_df["next6QRevenue"] = quarter_df["revenueAvg"].rolling(window=indexer, min_periods=1).sum()


quarter_df
    

revenueAvg       yq  next2QRevenue  next4QRevenue  next6QRevenue
0      2385250000  1996.2Q   4.809408e+09   9.597163e+09   1.498529e+10
1      2424158329  1996.3Q   4.735821e+09   9.803716e+09   1.441689e+10
2      2311662500  1996.4Q   4.787755e+09   1.017588e+10   1.345941e+10
3      2476092500  1997.1Q   5.067895e+09   9.681070e+09   1.273013e+10
4      2591802499  1997.2Q   5.388127e+09   8.671652e+09   1.196736e+10
..            ...      ...            ...            ...            ...
101   74484755402  2021.3Q   1.815667e+11   3.249744e+11   4.038395e+11
102  107081926524  2021.4Q   1.817275e+11   3.293548e+11   3.293548e+11
103   74645539765  2022.1Q   1.434077e+11   2.222728e+11   2.222728e+11
104   68762190142  2022.2Q   1.476273e+11   1.476273e+11   1.476273e+11
105   78865119080  2022.3Q   7.886512e+10   7.886512e+10   7.886512e+10

[106 rows x 5 columns]

In [7]:
merge_df = pd.merge(df,quarter_df, on='yq', how='left')
merge_df = merge_df.dropna()
merge_df = merge_df.drop("revenueAvg", axis=1)
merge_df

index  accountsPayable  accountsReceivables  accumulatedDepreciation  \
27      97              762                 1292                     -791   
28      96              791                 1496                     -750   
29      95              820                 1492                     -734   
30      94              840                 1149                     -739   
31      93              812                 1207                     -746   
..     ...              ...                  ...                      ...   
120      4            46236                22926                   -58579   
121      3            45111                20970                   -60959   
122      2            32421                15722                   -62854   
123      1            35325                17882                   -65230   
124      0            42296                16120                   -66760   

     cashEquivalents  cashShortTermInvestments  commonStock  currentAssets  \
27              1359                      1359          423           4454   
28              1552                      1745          439           4515   
29              1174                      1807          442           4419   
30              1273                      1459          472           3642   
31              1018                      1230          476           3493   
..               ...                       ...          ...            ...   
120            36640                    100557        45174         162819   
121            28388                    107162        45972         163231   
122            15769                     94051        48032         143753   
123            13130                     93025        48696         140065   
124            20243                     90943        50779         143713   

     currentLiabilities  inventory  liabilitiesShareholdersEquity  \
27                 1926       1061                           5345   
28                 2003        662                           5364   
29                 2044        488                           5272   
30                 2011        509                           4487   
31                 1910        534                           4341   
..                  ...        ...                            ...   
120              105718       4106                         338516   
121              102161       4097                         340618   
122               96094       3334                         320400   
123               95318       3978                         317344   
124              105392       4061                         323888   

     otherCurrentAssets  otherLongTermAssets  otherReceivables      period  \
27                  742                  276                 0  1996-06-28   
28                  612                  251                 0  1996-09-27   
29                  632                  260                 0  1996-12-27   
30                  525                  289                 0  1997-03-28   
31                  522                  308                 0  1997-06-27   
..                  ...                  ...               ...         ...   
120               12352                32978             22878  2019-09-28   
121               12026                33195             18976  2019-12-28   
122               15691                33868             14955  2020-03-28   
123               10987                32836             14193  2020-06-27   
124               11264                33952             21325  2020-09-26   

     propertyPlantEquipment  retainedEarnings  sharesOutstanding  \
27                      615              1609           3466.000   
28                      598              1634           3486.000   
29                      593              1514           3486.000   
30                      556               806           3540.000   
31                      540          

In [8]:
econ_data = finnhub_client.economic_data('MA-USA-656880')
econ_df = pd.DataFrame(econ_data["data"])
econ_df

date  value
0    2020-10-31    365
1    2020-09-30    753
2    2020-08-31    482
3    2020-07-31    216
4    2020-06-30   4485
..          ...    ...
230  2001-08-31   -219
231  2001-07-31   -168
232  2001-06-30   -172
233  2001-05-31   -186
234  2001-04-30   -207

[235 rows x 2 columns]

In [9]:
econ_df["year"] = pd.DatetimeIndex(econ_df['date']).year
econ_df.loc[(pd.DatetimeIndex(econ_df['date']).month == 1)|(pd.DatetimeIndex(econ_df['date']).month == 2)|(pd.DatetimeIndex(econ_df['date']).month == 3), 'quarter'] = '1Q'
econ_df.loc[(pd.DatetimeIndex(econ_df['date']).month == 4)|(pd.DatetimeIndex(econ_df['date']).month == 5)|(pd.DatetimeIndex(econ_df['date']).month == 6), 'quarter'] = '2Q'
econ_df.loc[(pd.DatetimeIndex(econ_df['date']).month == 7)|(pd.DatetimeIndex(econ_df['date']).month == 8)|(pd.DatetimeIndex(econ_df['date']).month == 9), 'quarter'] = '3Q'
econ_df.loc[(pd.DatetimeIndex(econ_df['date']).month == 10)|(pd.DatetimeIndex(econ_df['date']).month == 11)|(pd.DatetimeIndex(econ_df['date']).month == 12), 'quarter'] = '4Q'
econ_df["yq"] = econ_df["year"].map(str) + "." + econ_df["quarter"].map(str)

#econ_df = econ_df.drop(["year", "quarter"], axis=1)
econ_df


date  value  year quarter       yq
0    2020-10-31    365  2020      4Q  2020.4Q
1    2020-09-30    753  2020      3Q  2020.3Q
2    2020-08-31    482  2020      3Q  2020.3Q
3    2020-07-31    216  2020      3Q  2020.3Q
4    2020-06-30   4485  2020      2Q  2020.2Q
..          ...    ...   ...     ...      ...
230  2001-08-31   -219  2001      3Q  2001.3Q
231  2001-07-31   -168  2001      3Q  2001.3Q
232  2001-06-30   -172  2001      2Q  2001.2Q
233  2001-05-31   -186  2001      2Q  2001.2Q
234  2001-04-30   -207  2001      2Q  2001.2Q

[235 rows x 5 columns]

In [10]:
econgroup_df = econ_df[["yq","value"]].groupby(["yq"]).mean()
econgroup_df


value
yq                  
2001.2Q  -188.333333
2001.3Q  -203.333333
2001.4Q  -353.333333
2002.1Q   -96.666667
2002.2Q   -32.666667
...              ...
2019.4Q   133.666667
2020.1Q    16.666667
2020.2Q -3861.000000
2020.3Q   483.666667
2020.4Q   365.000000

[79 rows x 1 columns]

In [11]:
merge_df2 = pd.merge(merge_df,econgroup_df, on='yq', how='left')
merge_df2 = merge_df2.rename(columns={"value": "AvgEcon"})
merge_df2 = merge_df2.dropna()
merge_df2 = merge_df2.drop(["index"], axis = 1)
merge_df2

accountsPayable  accountsReceivables  accumulatedDepreciation  \
17              847                  598                     -380   
18              801                  466                     -396   
19              830                  498                     -361   
20              987                  644                     -368   
21              904                  626                     -381   
..              ...                  ...                      ...   
86            46236                22926                   -58579   
87            45111                20970                   -60959   
88            32421                15722                   -62854   
89            35325                17882                   -65230   
90            42296                16120                   -66760   

    cashEquivalents  cashShortTermInvestments  commonStock  currentAssets  \
17             2121                      4218         1673           5248   
18             2310                      4336         1693           5143   
19             1946                      4367         1706           5272   
20             1159                      4309         1747           5414   
21             1246                      4306         1808           5443   
..              ...                       ...          ...            ...   
86            36640                    100557        45174         162819   
87            28388                    107162        45972         163231   
88            15769                     94051        48032         143753   
89            13130                     93025        48696         140065   
90            20243                     90943        50779         143713   

    currentLiabilities  inventory  liabilitiesShareholdersEquity  \
17                1614         19                           6071   
18                1518         11                           6021   
19                1579         23                           6122   
20                1709         26                           6264   
21                1718         34                           6293   
..                 ...        ...                            ...   
86              105718       4106                         338516   
87              102161       4097                         340618   
88               96094       3334                         320400   
89               95318       3978                         317344   
90              105392       4061                         323888   

    otherCurrentAssets  otherLongTermAssets  otherReceivables      period  \
17                 413                  330                 0  2001-06-30   
18                 330                  110                 0  2001-09-29   
19                 384                  176                 0  2001-12-29   
20                 435                  103                 0  2002-03-30   
21                 477                  114                 0  2002-06-29   
..                 ...                  ...               ...         ...   
86               12352                32978             22878  2019-09-28   
87               12026                33195             18976  2019-12-28   
88               15691                33868             14955  2020-03-28   
89               10987                32836             14193  2020-06-27   
90               11264                33952             21325  2020-09-26   

    propertyPlantEquipment  retainedEarnings  sharesOutstanding  \
17                     347              2194           4907.000   
18                     564              2260           4913.000   
19                     573              2298           4934.000   
20                     587              2338           4957.000   
21                     589              2370           5024.000   
..                     ...               ...                ...   
86                   37378             

In [12]:
trend = finnhub_client.recommendation_trends('AAPL')
trend_df = pd.DataFrame(trend)
trend_df

buy  hold      period  sell  strongBuy  strongSell symbol
0    24     7  2020-11-01     0         13           0   AAPL
1    19    10  2020-10-01     2         13           2   AAPL
2    19    10  2020-09-01     2         13           2   AAPL
3    13    19  2020-08-01     3         15           0   AAPL
4    13    19  2020-07-01     3         15           0   AAPL
5    13    19  2020-06-01     3         15           0   AAPL
6    20     8  2020-05-01     3         14           0   AAPL
7    13    19  2020-04-01     3         15           0   AAPL
8    13    19  2020-03-01     3         15           0   AAPL
9    13    19  2020-02-01     3         15           0   AAPL
10   13    19  2020-01-01     3         15           0   AAPL
11   13    19  2019-12-01     3         15           0   AAPL
12   13    19  2019-11-01     3         15           0   AAPL
13   13    19  2019-10-01     3         15           0   AAPL
14   13    19  2019-09-01     3         15           0   AAPL
15   13    19  2019-08-01     3         15           0   AAPL
16   13    19  2019-07-01     3         15           0   AAPL
17   10    20  2019-06-01     2         15           0   AAPL
18   10    22  2019-05-01     1         15           0   AAPL
19   10    22  2019-04-01     1         15           0   AAPL
20   10    22  2019-03-01     1         15           0   AAPL
21   10    22  2019-02-01     1         15           0   AAPL
22   14    18  2019-01-01     0         17           0   AAPL
23   14    17  2018-12-01     0         16           0   AAPL
24   16    14  2018-11-01     0         19           0   AAPL
25   16    15  2018-10-01     0         17           0   AAPL
26   15    15  2018-09-01     0         17           0   AAPL
27   16    15  2018-08-01     0         15           0   AAPL
28   16    16  2018-07-01     0         15           0   AAPL
29   17    16  2018-06-01     0         15           0   AAPL
30   19    15  2018-05-01     0         15           0   AAPL
31   19    14  2018-04-01     0         13           0   AAPL

In [13]:
trend_df["year"] = pd.DatetimeIndex(trend_df['period']).year
trend_df.loc[(pd.DatetimeIndex(trend_df['period']).month == 1)|(pd.DatetimeIndex(trend_df['period']).month == 2)|(pd.DatetimeIndex(trend_df['period']).month == 3), 'quarter'] = '1Q'
trend_df.loc[(pd.DatetimeIndex(trend_df['period']).month == 4)|(pd.DatetimeIndex(trend_df['period']).month == 5)|(pd.DatetimeIndex(trend_df['period']).month == 6), 'quarter'] = '2Q'
trend_df.loc[(pd.DatetimeIndex(trend_df['period']).month == 7)|(pd.DatetimeIndex(trend_df['period']).month == 8)|(pd.DatetimeIndex(trend_df['period']).month == 9), 'quarter'] = '3Q'
trend_df.loc[(pd.DatetimeIndex(trend_df['period']).month == 10)|(pd.DatetimeIndex(trend_df['period']).month == 11)|(pd.DatetimeIndex(trend_df['period']).month == 12), 'quarter'] = '4Q'
trend_df["yq"] = trend_df["year"].map(str) + "." + trend_df["quarter"].map(str)

trend_df = trend_df.drop(["period", "year","quarter", "symbol"], axis = 1)

trend_df

buy  hold  sell  strongBuy  strongSell       yq
0    24     7     0         13           0  2020.4Q
1    19    10     2         13           2  2020.4Q
2    19    10     2         13           2  2020.3Q
3    13    19     3         15           0  2020.3Q
4    13    19     3         15           0  2020.3Q
5    13    19     3         15           0  2020.2Q
6    20     8     3         14           0  2020.2Q
7    13    19     3         15           0  2020.2Q
8    13    19     3         15           0  2020.1Q
9    13    19     3         15           0  2020.1Q
10   13    19     3         15           0  2020.1Q
11   13    19     3         15           0  2019.4Q
12   13    19     3         15           0  2019.4Q
13   13    19     3         15           0  2019.4Q
14   13    19     3         15           0  2019.3Q
15   13    19     3         15           0  2019.3Q
16   13    19     3         15           0  2019.3Q
17   10    20     2         15           0  2019.2Q
18   10    22     1         15           0  2019.2Q
19   10    22     1         15           0  2019.2Q
20   10    22     1         15           0  2019.1Q
21   10    22     1         15           0  2019.1Q
22   14    18     0         17           0  2019.1Q
23   14    17     0         16           0  2018.4Q
24   16    14     0         19           0  2018.4Q
25   16    15     0         17           0  2018.4Q
26   15    15     0         17           0  2018.3Q
27   16    15     0         15           0  2018.3Q
28   16    16     0         15           0  2018.3Q
29   17    16     0         15           0  2018.2Q
30   19    15     0         15           0  2018.2Q
31   19    14     0         13           0  2018.2Q

In [14]:
trendgroup_df = trend_df.groupby(["yq"]).mean()
trendgroup_df
## didn't use due to not having enough data

buy       hold      sell  strongBuy  strongSell
yq                                                            
2018.2Q  18.333333  15.000000  0.000000  14.333333    0.000000
2018.3Q  15.666667  15.333333  0.000000  15.666667    0.000000
2018.4Q  15.333333  15.333333  0.000000  17.333333    0.000000
2019.1Q  11.333333  20.666667  0.666667  15.666667    0.000000
2019.2Q  10.000000  21.333333  1.333333  15.000000    0.000000
2019.3Q  13.000000  19.000000  3.000000  15.000000    0.000000
2019.4Q  13.000000  19.000000  3.000000  15.000000    0.000000
2020.1Q  13.000000  19.000000  3.000000  15.000000    0.000000
2020.2Q  15.333333  15.333333  3.000000  14.666667    0.000000
2020.3Q  15.000000  16.000000  2.666667  14.333333    0.666667
2020.4Q  21.500000   8.500000  1.000000  13.000000    1.000000

In [15]:
url = "https://financialmodelingprep.com/api/v3/ratios/"
   
query_url = url + ticker + "?period=quarter&limit=1000" + "&apikey=" + API_key

ratio = requests.get(query_url).json()

print(ratio)

[{'symbol': 'AAPL', 'date': '2020-09-26', 'currentRatio': 1.3636044481554577, 'quickRatio': 1.0158550933657204, 'cashRatio': 0.36071049035979963, 'daysOfSalesOutstanding': 90.94253299947448, 'daysOfInventoryOutstanding': 9.135194581219226, 'operatingCycle': 31.559380800267732, 'daysOfPayablesOutstanding': 95.144592466695, 'cashConversionCycle': -63.58521166642727, 'grossProfitMargin': 0.3816037590033695, 'operatingProfitMargin': 0.2283687285542057, 'pretaxProfitMargin': 0.23031623852360197, 'netProfitMargin': 0.19587931620761073, 'effectiveTaxRate': 0.1495201664317831, 'returnOnAssets': 0.039127723163562715, 'returnOnEquity': 0.19395766693705138, 'returnOnCapitalEmployed': 0.06819804481546574, 'netIncomePerEBT': 0.8504798335682169, 'ebtPerEbit': 1.0, 'ebitPerRevenue': 0.23031623852360197, 'debtRatio': 0.7982666847799239, 'debtEquityRatio': 3.957039440456695, 'longTermDebtToCapitalization': 0.6016060388034584, 'totalDebtToCapitalization': 0.6218348294642289, 'interestCoverage': None, 'c

In [16]:
ratio_df = pd.DataFrame(ratio)

ratio_df["year"] = pd.DatetimeIndex(ratio_df['date']).year
ratio_df.loc[(pd.DatetimeIndex(ratio_df['date']).month == 1)|(pd.DatetimeIndex(ratio_df['date']).month == 2)|(pd.DatetimeIndex(ratio_df['date']).month == 3), 'quarter'] = '1Q'
ratio_df.loc[(pd.DatetimeIndex(ratio_df['date']).month == 4)|(pd.DatetimeIndex(ratio_df['date']).month == 5)|(pd.DatetimeIndex(ratio_df['date']).month == 6), 'quarter'] = '2Q'
ratio_df.loc[(pd.DatetimeIndex(ratio_df['date']).month == 7)|(pd.DatetimeIndex(ratio_df['date']).month == 8)|(pd.DatetimeIndex(ratio_df['date']).month == 9), 'quarter'] = '3Q'
ratio_df.loc[(pd.DatetimeIndex(ratio_df['date']).month == 10)|(pd.DatetimeIndex(ratio_df['date']).month == 11)|(pd.DatetimeIndex(ratio_df['date']).month == 12), 'quarter'] = '4Q'
ratio_df["yq"] = ratio_df["year"].map(str) + "." + ratio_df["quarter"].map(str)


ratio_df = ratio_df.drop(["symbol" , "date", "year", "quarter"], axis = 1)
ratio_df = ratio_df.dropna()

ratio_df

currentRatio  quickRatio  cashRatio  daysOfSalesOutstanding  \
1       1.469450    1.312449   0.350228              196.152718   
2       1.495962    1.297979   0.418070              192.017303   
3       1.597782    1.439962   0.389297              158.793823   
4       1.540126    1.384447   0.462022              261.062773   
5       1.504649    1.349862   0.563297              179.579810   
6       1.315382    1.134347   0.405110              165.327415   
7       1.300555    1.139680   0.413463              160.100403   
8       1.132926    0.994540   0.223525              284.311208   
9       1.307325    1.099257   0.361058              180.680653   
10      1.456034    1.235423   0.504467              133.780199   
11      1.242011    1.105918   0.237425              210.414586   
12      1.276063    1.089670   0.201252              247.639647   
13      1.388342    1.222491   0.228420              181.921247   
14      1.390608    1.195945   0.206662              142.229658   
15      1.228242    1.051099   0.194592              130.331521   
16      1.352669    1.220844   0.259271              228.253543   
17      1.311599    1.130263   0.255113              164.085415   
18      1.283117    1.100227   0.315154              143.120834   
19      1.001669    0.823937   0.219327              118.445079   
20      1.108771    0.892495   0.262002              215.048154   
21      1.086819    0.836486   0.234648              146.478278   
22      1.156005    0.872823   0.246709              114.288226   
23      1.133024    0.848229   0.264607              146.667382   
24      1.080113    0.824234   0.218194              235.855352   
25      1.470598    1.182686   0.280857              164.216847   

    daysOfInventoryOutstanding  operatingCycle  daysOfPayablesOutstanding  \
1                     9.674909       58.041333                  85.914066   
2                     8.348218       55.694950                  81.181037   
3                     6.514434       45.669075                  71.728738   
4                     9.301986       73.673629                 104.745891   
5                     8.991424       53.271377                  78.028408   
6                    12.144554       52.910218                  75.699563   
7                     8.587004       48.063816                  76.251841   
8                     9.172506       79.276640                 129.583677   
9                    16.265985       60.817379                 105.468579   
10                   18.283972       51.270870                  81.876972   
11                    7.316710       59.199758                 104.239532   
12                   13.383668       74.445499                 121.960916   
13                   10.141117       54.998411                 102.877865   
14                    8.107104       43.177431                  79.602848   
15                    5.066528       37.203068                  71.943954   
16                    6.607666       62.889361                 115.584559   
17                    6.277236       46.736653                  90.226268   
18                    6.700940       41.991009                  73.730905   
19                    4.853572       34.059208                  65.965808   
20                    6.830033       59.855605                 103.191936   
21                    6.141559       42.259490                  79.623713   
22                    6.276998       34.457657                  60.671538   
23                    4.580454       40.745014                  76.242588   
24                    7.275408       65.431522                 104.068316   
25                    6.320659       46.812484                  81.427061   

    cashConversionCycle  grossProfitMargin  operatingProfitMargin  \
1            -27.872733           0.379995               0.219335   
2            -25.486086           0.383619               0.220414   
3            -26.059662           0.383548         

In [17]:
merge_df3 = pd.merge(merge_df2,ratio_df, on='yq', how='left')

merge_df3 = merge_df3.dropna()
merge_df3.tail()

accountsPayable  accountsReceivables  accumulatedDepreciation  \
69            29115                14148                   -56348   
70            46236                22926                   -58579   
71            45111                20970                   -60959   
72            32421                15722                   -62854   
73            35325                17882                   -65230   

    cashEquivalents  cashShortTermInvestments  commonStock  currentAssets  \
69            38455                     94614        43371         134973   
70            36640                    100557        45174         162819   
71            28388                    107162        45972         163231   
72            15769                     94051        48032         143753   
73            13130                     93025        48696         140065   

    currentLiabilities  inventory  liabilitiesShareholdersEquity  \
69               89704       3355                         322239   
70              105718       4106                         338516   
71              102161       4097                         340618   
72               96094       3334                         320400   
73               95318       3978                         317344   

    otherCurrentAssets  otherLongTermAssets  otherReceivables      period  \
69               10530                33634             12326  2019-06-29   
70               12352                32978             22878  2019-09-28   
71               12026                33195             18976  2019-12-28   
72               15691                33868             14955  2020-03-28   
73               10987                32836             14193  2020-06-27   

    propertyPlantEquipment  retainedEarnings  sharesOutstanding  \
69                   37636             53724          18125.580   
70                   37378             45898          17772.944   
71                   44293             43977          17539.836   
72                   43986             33182          17295.948   
73                   43851             24136          17135.756   

    shortTermDebt  shortTermInvestments  tangibleBookValueperShare  \
69           9953                 44084                    5.32154   
70           5980                 51713                    5.09133   
71           4990                 67391                    5.10444   
72          10029                 53877                    4.53430   
73          11166                 59642                    4.21820   

    totalAssets  totalDebt  totalEquity  totalLiabilities  totalLongTermDebt  \
69       322239     108418        96456            225783              84936   
70       338516     108047        90488            248028              91807   
71       340618     108933        89531            251087              93705   
72       320400     110155        78425            241975              89715   
73       317344     113373        72282            245062              94678   

    totalReceivables  unrealizedProfitLossSecurity  year       yq  \
69             26474                           328  2019  2019.2Q   
70             45804                           707  2019  2019.3Q   
71             39946                           822  2019  2019.4Q   
72             30677                         -1474  2020  2020.1Q   
73             32075                          1613  2020  2020.2Q   

    next2QRevenue  next4QRevenue  next6QRevenue      AvgEcon  currentRatio  \
69   1.181230e+11   2.633094e+11   3.807949e+11   150.000000      1.504649   
70   1.537538e+11   2.621478e+11   4.289874e+11   160.000000      1.540126   
71   1.451864e+11   2.626719e+11   4.355936e+11   133.666667      1.597782   
72   1.083940e+11   2.752337e+11   4.111009e+11    16.666667      1.495962   
73   1.174855e+11   2.904072e+11   4.302231e+11 -3861.000000      1.469450   

    quickRatio  cashRatio  daysOfSalesOutstanding  daysOfInventor

In [18]:
url2 = "https://financialmodelingprep.com/api/v3/income-statement/"

   
query_url = url2 + ticker + "?period=quarter&limit=1000" + "&apikey=" + API_key

income = requests.get(query_url).json()

print(income)


[{'date': '2020-09-26', 'symbol': 'AAPL', 'fillingDate': '2020-10-30', 'acceptedDate': '2020-10-29 18:06:25', 'period': 'Q4', 'revenue': 64698000000, 'costOfRevenue': 40009000000, 'grossProfit': 24689000000, 'grossProfitRatio': 0.38160375900337, 'researchAndDevelopmentExpenses': 4978000000, 'generalAndAdministrativeExpenses': 4936000000, 'sellingAndMarketingExpenses': 0.0, 'otherExpenses': 126000000, 'operatingExpenses': 9914000000, 'costAndExpenses': 49923000000, 'interestExpense': 0.0, 'depreciationAndAmortization': 2702000000, 'ebitda': 17477000000, 'ebitdaratio': 0.270131997897926, 'operatingIncome': 14775000000, 'operatingIncomeRatio': 0.228368728554206, 'totalOtherIncomeExpensesNet': 126000000, 'incomeBeforeTax': 14901000000, 'incomeBeforeTaxRatio': 0.230316238523602, 'incomeTaxExpense': 2228000000, 'netIncome': 12673000000, 'netIncomeRatio': 0.195879316207611, 'eps': 0.73, 'epsdiluted': 0.73, 'weightedAverageShsOut': 17352119000, 'weightedAverageShsOutDil': 17528214000, 'link': 

In [19]:
income_df = pd.DataFrame(income)

income_df = income_df.dropna()
income_df["year"] = pd.DatetimeIndex(income_df['date']).year
income_df = income_df.astype({'year': 'int64'})
income_df.loc[(pd.DatetimeIndex(income_df['date']).month == 1)|(pd.DatetimeIndex(income_df['date']).month == 2)|(pd.DatetimeIndex(income_df['date']).month == 3), 'quarter'] = '1Q'
income_df.loc[(pd.DatetimeIndex(income_df['date']).month == 4)|(pd.DatetimeIndex(income_df['date']).month == 5)|(pd.DatetimeIndex(income_df['date']).month == 6), 'quarter'] = '2Q'
income_df.loc[(pd.DatetimeIndex(income_df['date']).month == 7)|(pd.DatetimeIndex(income_df['date']).month == 8)|(pd.DatetimeIndex(income_df['date']).month == 9), 'quarter'] = '3Q'
income_df.loc[(pd.DatetimeIndex(income_df['date']).month == 10)|(pd.DatetimeIndex(income_df['date']).month == 11)|(pd.DatetimeIndex(income_df['date']).month == 12), 'quarter'] = '4Q'

income_df["yq"] = income_df["year"].map(str) + "." + income_df["quarter"].map(str)


income_df = income_df.drop(["symbol" , "fillingDate", "acceptedDate", "date", "period", "year", "quarter", "link", "finalLink"], axis = 1)
income_df = income_df.dropna()

income_df

revenue  costOfRevenue  grossProfit  grossProfitRatio  \
0    64698000000    40009000000  24689000000          0.381604   
1    59685000000    37005000000  22680000000          0.379995   
2    58313000000    35943000000  22370000000          0.383619   
3    91819000000    56602000000  35217000000          0.383548   
4    64040000000    39727000000  24313000000          0.379653   
..           ...            ...          ...               ...   
101   2575000000     1847000000    728000000          0.282718   
102   2652000000     1957000000    695000000          0.262066   
103   2832000000     2018000000    814000000          0.287429   
104   2493000000     1814000000    679000000          0.272363   
105   2150000000     1576000000    574000000          0.266977   

     researchAndDevelopmentExpenses  generalAndAdministrativeExpenses  \
0                        4978000000                        4936000000   
1                        4758000000                        4831000000   
2                        4565000000                        4952000000   
3                        4451000000                        5197000000   
4                        4110000000                        4578000000   
..                              ...                               ...   
101                       168000000                         404000000   
102                       143000000                         386000000   
103                       132000000                         415000000   
104                       142000000                         346000000   
105                       135000000                         333000000   

     sellingAndMarketingExpenses  otherExpenses  operatingExpenses  \
0                            0.0    126000000.0         9914000000   
1                            0.0     46000000.0         9589000000   
2                            0.0    282000000.0         9517000000   
3                            0.0    349000000.0         9648000000   
4                            0.0    502000000.0         8688000000   
..                           ...            ...                ...   
101                          0.0            0.0          566000000   
102                          0.0            0.0          529000000   
103                          0.0            0.0          530000000   
104                          0.0            0.0          489000000   
105                          0.0            0.0          341000000   

     costAndExpenses  interestExpense  depreciationAndAmortization  \
0        49923000000              0.0                 2.702000e+09   
1        46594000000      697000000.0                 2.752000e+09   
2        45460000000      757000000.0                 2.786000e+09   
3        66250000000      785000000.0                 2.816000e+09   
4        48415000000      810000000.0                 3.179000e+09   
..               ...              ...                          ...   
101       2413000000              0.0                 3.700000e+07   
102       2486000000              0.0                 2.900000e+07   
103       2548000000              0.0                 3.800000e+07   
104       2303000000              0.0                 4.600000e+07   
105       1917000000              0.0                 4.100000e+07   

          ebitda  ebitdaratio  operatingIncome  operatingIncomeRatio  \
0    17477000000     0.270132     1.477500e+10              0.228369   
1    16586000000     0.277892     1.309100e+10              0.219335   
2    16678000000     0.286008     1.285300e+10              0.220414   
3    29519000000     0.321491     2.556900e+10              0.278472   
4    20116000000     0.314116     1.562500e+10              0.243988   
..           ...          ...              ...                   ...   
101    201000000     0.078058     1.620000e+08              0.062913   
102    145000000     0.054676     1.660000e+08              0.062594   
103    33

In [20]:
merge_df4 = pd.merge(merge_df3,income_df, on='yq', how='left')

merge_df4 = merge_df4.dropna()


#merge_df4[["period", "yq", "priceToSalesRatio", "revenue", "marketCap", "sharesOutstanding", "sharePrice"]].tail()
merge_df4.tail()

accountsPayable  accountsReceivables  accumulatedDepreciation  \
21            29115                14148                   -56348   
22            46236                22926                   -58579   
23            45111                20970                   -60959   
24            32421                15722                   -62854   
25            35325                17882                   -65230   

    cashEquivalents  cashShortTermInvestments  commonStock  currentAssets  \
21            38455                     94614        43371         134973   
22            36640                    100557        45174         162819   
23            28388                    107162        45972         163231   
24            15769                     94051        48032         143753   
25            13130                     93025        48696         140065   

    currentLiabilities  inventory  liabilitiesShareholdersEquity  \
21               89704       3355                         322239   
22              105718       4106                         338516   
23              102161       4097                         340618   
24               96094       3334                         320400   
25               95318       3978                         317344   

    otherCurrentAssets  otherLongTermAssets  otherReceivables      period  \
21               10530                33634             12326  2019-06-29   
22               12352                32978             22878  2019-09-28   
23               12026                33195             18976  2019-12-28   
24               15691                33868             14955  2020-03-28   
25               10987                32836             14193  2020-06-27   

    propertyPlantEquipment  retainedEarnings  sharesOutstanding  \
21                   37636             53724          18125.580   
22                   37378             45898          17772.944   
23                   44293             43977          17539.836   
24                   43986             33182          17295.948   
25                   43851             24136          17135.756   

    shortTermDebt  shortTermInvestments  tangibleBookValueperShare  \
21           9953                 44084                    5.32154   
22           5980                 51713                    5.09133   
23           4990                 67391                    5.10444   
24          10029                 53877                    4.53430   
25          11166                 59642                    4.21820   

    totalAssets  totalDebt  totalEquity  totalLiabilities  totalLongTermDebt  \
21       322239     108418        96456            225783              84936   
22       338516     108047        90488            248028              91807   
23       340618     108933        89531            251087              93705   
24       320400     110155        78425            241975              89715   
25       317344     113373        72282            245062              94678   

    totalReceivables  unrealizedProfitLossSecurity  year       yq  \
21             26474                           328  2019  2019.2Q   
22             45804                           707  2019  2019.3Q   
23             39946                           822  2019  2019.4Q   
24             30677                         -1474  2020  2020.1Q   
25             32075                          1613  2020  2020.2Q   

    next2QRevenue  next4QRevenue  next6QRevenue      AvgEcon  currentRatio  \
21   1.181230e+11   2.633094e+11   3.807949e+11   150.000000      1.504649   
22   1.537538e+11   2.621478e+11   4.289874e+11   160.000000      1.540126   
23   1.451864e+11   2.626719e+11   4.355936e+11   133.666667      1.597782   
24   1.083940e+11   2.752337e+11   4.111009e+11    16.666667      1.495962   
25   1.174855e+11   2.904072e+11   4.302231e+11 -3861.000000      1.469450   

    quickRatio  cashRatio  daysOfSalesOutstanding  daysOfInventor

In [21]:
url3 = "https://financialmodelingprep.com/api/v3/historical-price-full/"

   
query_url = url3 + ticker + "?apikey=" + API_key

price = requests.get(query_url).json()

print(price)


{'symbol': 'AAPL', 'historical': [{'date': '2020-11-09', 'open': 120.5, 'high': 121.985001, 'low': 116.050003, 'close': 116.32, 'adjClose': 116.32, 'volume': 154515315.0, 'unadjustedVolume': 154515315.0, 'change': -4.18, 'changePercent': -3.469, 'vwap': 118.11833, 'label': 'November 09, 20', 'changeOverTime': -0.03469}, {'date': '2020-11-06', 'open': 118.32, 'high': 119.199997, 'low': 116.129997, 'close': 118.690002, 'adjClose': 118.690002, 'volume': 114283600.0, 'unadjustedVolume': 114283600.0, 'change': 0.37, 'changePercent': 0.313, 'vwap': 118.00667, 'label': 'November 06, 20', 'changeOverTime': 0.00313}, {'date': '2020-11-05', 'open': 117.949997, 'high': 119.620003, 'low': 116.870003, 'close': 119.029999, 'adjClose': 118.824997, 'volume': 126387100.0, 'unadjustedVolume': 126387100.0, 'change': 1.08, 'changePercent': 0.916, 'vwap': 118.50667, 'label': 'November 05, 20', 'changeOverTime': 0.00916}, {'date': '2020-11-04', 'open': 114.139999, 'high': 115.589996, 'low': 112.349998, 'clo

In [22]:
price_df = pd.DataFrame(price["historical"])

price_df["year"] = pd.DatetimeIndex(price_df['date']).year
price_df = price_df.astype({'year': 'int64'})
price_df.loc[(pd.DatetimeIndex(price_df['date']).month == 1)|(pd.DatetimeIndex(price_df['date']).month == 2)|(pd.DatetimeIndex(price_df['date']).month == 3), 'quarter'] = '1Q'
price_df.loc[(pd.DatetimeIndex(price_df['date']).month == 4)|(pd.DatetimeIndex(price_df['date']).month == 5)|(pd.DatetimeIndex(price_df['date']).month == 6), 'quarter'] = '2Q'
price_df.loc[(pd.DatetimeIndex(price_df['date']).month == 7)|(pd.DatetimeIndex(price_df['date']).month == 8)|(pd.DatetimeIndex(price_df['date']).month == 9), 'quarter'] = '3Q'
price_df.loc[(pd.DatetimeIndex(price_df['date']).month == 10)|(pd.DatetimeIndex(price_df['date']).month == 11)|(pd.DatetimeIndex(price_df['date']).month == 12), 'quarter'] = '4Q'

price_df["yq"] = price_df["year"].map(str) + "." + price_df["quarter"].map(str)


price_df

date        open        high         low       close    adjClose  \
0     2020-11-09  120.500000  121.985001  116.050003  116.320000  116.320000   
1     2020-11-06  118.320000  119.199997  116.129997  118.690002  118.690002   
2     2020-11-05  117.949997  119.620003  116.870003  119.029999  118.824997   
3     2020-11-04  114.139999  115.589996  112.349998  114.949997  114.752022   
4     2020-11-03  109.660004  111.489998  108.730003  110.440002  110.249794   
...          ...         ...         ...         ...         ...         ...   
1254  2015-11-16   27.844999   28.559999   27.750000   28.545000   26.491756   
1255  2015-11-13   28.799999   28.892500   28.067499   28.084999   26.064838   
1256  2015-11-12   29.065001   29.205000   28.912500   28.930000   26.849056   
1257  2015-11-11   29.092501   29.355000   28.802500   29.027500   26.939545   
1258  2015-11-10   29.225000   29.517500   29.014999   29.192499   27.092672   

           volume  unadjustedVolume  change  changePercent       vwap  \
0     154515315.0       154515315.0 -4.1800         -3.469  118.11833   
1     114283600.0       114283600.0  0.3700          0.313  118.00667   
2     126387100.0       126387100.0  1.0800          0.916  118.50667   
3     138235500.0       138235500.0  0.8100          0.710  114.29666   
4     107624400.0       107624400.0  0.7800          0.711  110.22000   
...           ...               ...     ...            ...        ...   
1254  152426800.0       152426800.0  0.7000          2.514   28.28500   
1255  183249600.0       183249600.0 -0.7150         -2.483   28.34833   
1256  130102400.0       130102400.0 -0.1350         -0.464   29.01583   
1257  180872000.0       180872000.0 -0.0650         -0.223   29.06167   
1258  236511600.0       236511600.0 -0.0325         -0.111   29.24167   

                label  changeOverTime  year quarter       yq  
0     November 09, 20        -0.03469  2020      4Q  2020.4Q  
1     November 06, 20         0.00313  2020      4Q  2020.4Q  
2     November 05, 20         0.00916  2020      4Q  2020.4Q  
3     November 04, 20         0.00710  2020      4Q  2020.4Q  
4     November 03, 20         0.00711  2020      4Q  2020.4Q  
...               ...             ...   ...     ...      ...  
1254  November 16, 15         0.02514  2015      4Q  2015.4Q  
1255  November 13, 15        -0.02483  2015      4Q  2015.4Q  
1256  November 12, 15        -0.00464  2015      4Q  2015.4Q  
1257  November 11, 15        -0.00223  2015      4Q  2015.4Q  
1258  November 10, 15        -0.00111  2015      4Q  2015.4Q  

[1259 rows x 16 columns]

In [23]:
pricegroup_df = price_df.groupby(["yq"]).mean()
pricegroup_df = pricegroup_df.reset_index()
pricegroup_df = pricegroup_df.drop(["change", "changePercent", "high", "low", "changeOverTime", "unadjustedVolume", "open", "close", "vwap", "year"], axis = 1)



#pricegroup_df["change"] = pricegroup_df["adjClose"].rolling(2).apply(lambda x: x.iloc[1] - x.iloc[0]) 


pricegroup_df

yq    adjClose        volume
0   2015.4Q   26.378708  1.603868e+08
1   2016.1Q   23.202656  1.854925e+08
2   2016.2Q   23.273124  1.595346e+08
3   2016.3Q   24.935583  1.427452e+08
4   2016.4Q   26.844929  1.280788e+08
5   2017.1Q   31.322382  1.096593e+08
6   2017.2Q   35.309441  1.086644e+08
7   2017.3Q   37.200210  1.119654e+08
8   2017.4Q   40.217544  1.038818e+08
9   2018.1Q   41.591851  1.509050e+08
10  2018.2Q   43.989713  1.134222e+08
11  2018.3Q   50.701835  1.125703e+08
12  2018.4Q   47.350497  1.682538e+08
13  2019.1Q   41.609333  1.279744e+08
14  2019.2Q   47.976939  1.117964e+08
15  2019.3Q   51.682832  1.060938e+08
16  2019.4Q   63.816632  1.033646e+08
17  2020.1Q   73.136028  1.665009e+08
18  2020.2Q   77.273813  1.478715e+08
19  2020.3Q  109.043782  1.715138e+08
20  2020.4Q  116.024887  1.306761e+08

In [24]:
merge_df5 = pd.merge(merge_df4,pricegroup_df, on='yq', how='left')

merge_df5 = merge_df5.dropna()


merge_df5.tail()

accountsPayable  accountsReceivables  accumulatedDepreciation  \
21            29115                14148                   -56348   
22            46236                22926                   -58579   
23            45111                20970                   -60959   
24            32421                15722                   -62854   
25            35325                17882                   -65230   

    cashEquivalents  cashShortTermInvestments  commonStock  currentAssets  \
21            38455                     94614        43371         134973   
22            36640                    100557        45174         162819   
23            28388                    107162        45972         163231   
24            15769                     94051        48032         143753   
25            13130                     93025        48696         140065   

    currentLiabilities  inventory  liabilitiesShareholdersEquity  \
21               89704       3355                         322239   
22              105718       4106                         338516   
23              102161       4097                         340618   
24               96094       3334                         320400   
25               95318       3978                         317344   

    otherCurrentAssets  otherLongTermAssets  otherReceivables      period  \
21               10530                33634             12326  2019-06-29   
22               12352                32978             22878  2019-09-28   
23               12026                33195             18976  2019-12-28   
24               15691                33868             14955  2020-03-28   
25               10987                32836             14193  2020-06-27   

    propertyPlantEquipment  retainedEarnings  sharesOutstanding  \
21                   37636             53724          18125.580   
22                   37378             45898          17772.944   
23                   44293             43977          17539.836   
24                   43986             33182          17295.948   
25                   43851             24136          17135.756   

    shortTermDebt  shortTermInvestments  tangibleBookValueperShare  \
21           9953                 44084                    5.32154   
22           5980                 51713                    5.09133   
23           4990                 67391                    5.10444   
24          10029                 53877                    4.53430   
25          11166                 59642                    4.21820   

    totalAssets  totalDebt  totalEquity  totalLiabilities  totalLongTermDebt  \
21       322239     108418        96456            225783              84936   
22       338516     108047        90488            248028              91807   
23       340618     108933        89531            251087              93705   
24       320400     110155        78425            241975              89715   
25       317344     113373        72282            245062              94678   

    totalReceivables  unrealizedProfitLossSecurity  year       yq  \
21             26474                           328  2019  2019.2Q   
22             45804                           707  2019  2019.3Q   
23             39946                           822  2019  2019.4Q   
24             30677                         -1474  2020  2020.1Q   
25             32075                          1613  2020  2020.2Q   

    next2QRevenue  next4QRevenue  next6QRevenue      AvgEcon  currentRatio  \
21   1.181230e+11   2.633094e+11   3.807949e+11   150.000000      1.504649   
22   1.537538e+11   2.621478e+11   4.289874e+11   160.000000      1.540126   
23   1.451864e+11   2.626719e+11   4.355936e+11   133.666667      1.597782   
24   1.083940e+11   2.752337e+11   4.111009e+11    16.666667      1.495962   
25   1.174855e+11   2.904072e+11   4.302231e+11 -3861.000000      1.469450   

    quickRatio  cashRatio  daysOfSalesOutstanding  daysOfInventor

In [25]:
test_df = merge_df5
test_df = test_df.reset_index()
test_df = test_df.drop(["index", "year", "yq"], axis=1)
test_df = test_df.astype({'period': 'datetime64'} )

test_df

accountsPayable  accountsReceivables  accumulatedDepreciation  \
0             33312                12953                   -29042   
1             25098                12229                   -30848   
2             26318                11714                   -32543   
3             37294                15754                   -34235   
4             38510                14057                   -36249   
5             44242                17874                   -41293   
6             44242                17874                   -41293   
7             44242                17874                   -41293   
8             44242                17874                   -41293   
9             62985                23440                   -43431   
10            34311                14324                   -45425   
11            38489                14104                   -47251   
12            55888                23186                   -49099   
13            44293                18077                   -51929   
14            30443                15085                   -54290   
15            29115                14148                   -56348   
16            46236                22926                   -58579   
17            45111                20970                   -60959   
18            32421                15722                   -62854   
19            35325                17882                   -65230   

    cashEquivalents  cashShortTermInvestments  commonStock  currentAssets  \
0              5537                     38074        28253          76219   
1             11315                     55283        29484          87592   
2              9119                     61756        30106          93761   
3             11883                     67155        31251         106869   
4              7012                     60452        32144         103332   
5             12307                     74181        35867         128645   
6             12307                     74181        35867         128645   
7             12307                     74181        35867         128645   
8             12307                     74181        35867         128645   
9             17962                     77153        36447         143810   
10            35125                     87940        38044         130053   
11            21998                     70970        38624         115761   
12            14338                     66301        40201         131339   
13            35282                     86427        40970         140828   
14            26237                     80092        42801         123346   
15            38455                     94614        43371         134973   
16            36640                    100557        45174         162819   
17            28388                    107162        45972         163231   
18            15769                     94051        48032         143753   
19            13130                     93025        48696         140065   

    currentLiabilities  inventory  liabilitiesShareholdersEquity  \
0                76092       2451                         293284   
1                68265       2281                         305277   
2                71486       1831                         305602   
3                79006       2132                         321686   
4                84130       2712                         331141   
5               100814       4855                         375319   
6               100814       4855                         375319   
7               100814       4855                         375319   
8               100814       4855                         375319   
9               115788       4421                         406794   
10               89320       7662                         367502   
11               88548       5936                         349197   
12              115929       3956                   

In [26]:
#retrieving the number of rows
test_df.shape

(20, 113)

In [27]:
# add economic data + inflation?

#finnhub_client.recommendation_trends('AAPL')

#add financial ratio

#market cap - https://financialmodelingprep.com/developer/docs/#Company-Financial-Ratios

#Quarterly income statements- revenue
#Company Financial Ratios - priceToSalesRatio
#cap = PSratio*revenue
#calculate % depending on market cap

#cap/shares = stock price